In [7]:
import pandas as pd

In [10]:
telemetry1 = pd.read_csv("telemetry_1.csv")
telemetry2 = pd.read_csv("telemetry_2.csv")
vehicle_data = pd.read_csv("vehicle_data.csv")

## Display the first few rows of each dataset to understand the structure

In [11]:
print("Telemetry 1:\n", telemetry1.head())
print("\nTelemetry 2:\n", telemetry2.head())
print("\nVehicle Data:\n", vehicle_data.head())

Telemetry 1:
    vehicle_id                         timestamp  speed  odometer  fuel_level
0   vehicle_4         2024-06-06 14:26:23-07:00   64.2       NaN         NaN
1   vehicle_8  2024-07-05 22:24:12.856000-07:00    1.6   12048.0         NaN
2   vehicle_3  2024-06-20 12:57:59.306000-07:00    NaN       NaN        45.0
3  vehicle_10  2024-07-24 18:04:23.776000-07:00   36.2    1434.7         NaN
4  vehicle_11         2024-07-06 11:08:03-07:00   29.5       NaN         NaN

Telemetry 2:
    vehicle_id                         timestamp   val   name
0   vehicle_4         2024-07-23 19:56:44+05:30   0.0  speed
1  vehicle_11         2024-04-04 22:33:56+05:30  52.7  speed
2   vehicle_3  2024-07-18 23:30:04.027000+05:30   5.8  speed
3   vehicle_3  2024-05-22 00:49:49.402000+05:30  34.4  speed
4  vehicle_10  2024-07-16 05:48:11.710000+05:30  42.8  speed

Vehicle Data:
           id datasource  tank_capacity  rated_mpg
0  vehicle_1   source_1           18.0         35
1  vehicle_2   source_1    

# Normalize Telemetry 2 by pivoting so that the 'Name' column becomes the new columns

In [16]:
telemetry2_pivot = telemetry2.pivot_table(index=['vehicle_id', 'timestamp'], columns='name', values='val').reset_index()